In [ ]:
# This will list all files in the current directory
# You should see: studentVle.csv, studentAssessment.csv, studentInfo.csv, studentRegistration.csv

import os

print(os.listdir())  # list files in the current working directory


['.config', 'studentVle.csv', '.ipynb_checkpoints', 'studentRegistration.csv', 'studentAssessment.csv', 'studentInfo.csv', 'drive', 'sample_data']


In [ ]:
# Step 2: Import pandas and load the four main CSV files

import pandas as pd  # pandas is used for working with tables (dataframes)

# Load each CSV into a pandas DataFrame.
# We pass na_values=["?"] just in case the dataset uses "?" to represent missing values.

student_vle = pd.read_csv("studentVle.csv", na_values=["?"])
student_assessment = pd.read_csv("studentAssessment.csv", na_values=["?"])
student_info = pd.read_csv("studentInfo.csv", na_values=["?"])
student_reg = pd.read_csv("studentRegistration.csv", na_values=["?"])

# Print the number of rows and columns in each file to confirm they loaded correctly
print("Shapes of loaded tables:")
print(" studentVle        :", student_vle.shape)
print(" studentAssessment :", student_assessment.shape)
print(" studentInfo       :", student_info.shape)
print(" studentRegistration:", student_reg.shape)

# Show the first 5 rows of each, just to visually confirm the content
print("\nstudentVle head:")
display(student_vle.head())

print("\nstudentAssessment head:")
display(student_assessment.head())

print("\nstudentInfo head:")
display(student_info.head())

print("\nstudentRegistration head:")
display(student_reg.head())


Shapes of loaded tables:
 studentVle        : (10655280, 6)
 studentAssessment : (173912, 5)
 studentInfo       : (32593, 12)
 studentRegistration: (32593, 5)

studentVle head:


,code_module,code_presentation,id_student,id_site,date,sum_click
0,AAA,2013J,28400,546652,-10,4
1,AAA,2013J,28400,546652,-10,1
2,AAA,2013J,28400,546652,-10,1
3,AAA,2013J,28400,546614,-10,11
4,AAA,2013J,28400,546714,-10,1



studentAssessment head:


,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0



studentInfo head:


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass



studentRegistration head:


,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [ ]:
# Step 3: Build engagement features from studentVle
# One row per student with summary stats of their LMS activity

# We group by 'id_student' because we want one row per student.
engagement = (
    student_vle
    .groupby("id_student")  # group all rows belonging to the same student
    .agg(
        total_clicks=("sum_click", "sum"),      # total number of clicks across all days
        active_days=("date", "nunique"),        # number of unique days the student was active
        avg_daily_clicks=("sum_click", "mean"), # average clicks per active day
        first_active_day=("date", "min"),       # first day they interacted with LMS
        last_active_day=("date", "max"),        # last day they interacted with LMS
    )
    .reset_index()  # convert the groupby result back into a normal dataframe
)

print("Engagement features shape:", engagement.shape)
display(engagement.head())


Engagement features shape: (26074, 6)


,id_student,total_clicks,active_days,avg_daily_clicks,first_active_day,last_active_day
0,6516,2791,159,4.216012,-23,269
1,8462,656,56,2.157895,-6,118
2,11391,934,40,4.765306,-5,253
3,23629,161,16,2.728814,-6,87
4,23698,910,70,2.983607,-18,231


In [ ]:
# Step 4: Build performance features from studentAssessment
# One row per student with summary of their assessment behavior

performance = (
    student_assessment
    .groupby("id_student")  # again, one row per student
    .agg(
        quizzes_attempted=("score", "count"),  # how many assessments they attempted
        avg_quiz_score=("score", "mean"),      # average score across all assessments
    )
    .reset_index()
)

print("Performance features shape:", performance.shape)
display(performance.head())


Performance features shape: (23369, 3)


,id_student,quizzes_attempted,avg_quiz_score
0,6516,5,61.800000
1,8462,7,87.000000
2,11391,5,82.000000
3,23629,4,82.500000
4,23698,9,74.444444


In [ ]:
# Step 5: Build registration features from studentRegistration
# These tell us when the student registered and whether they unregistered (dropped out)

registration = (
    student_reg
    .groupby("id_student")
    .agg(
        date_registration=("date_registration", "min"),  # first day they registered
        date_unregistration=("date_unregistration", "max"),  # last unregistration day (if any)
    )
    .reset_index()
)

# Create a flag: 1 if the student has an unregistration date (means they withdrew), else 0
registration["unregistered_flag"] = registration["date_unregistration"].notna().astype(int)

print("Registration features shape:", registration.shape)
display(registration.head())


Registration features shape: (28785, 4)


,id_student,date_registration,date_unregistration,unregistered_flag
0,3733,-68.0,-8.0,1
1,6516,-52.0,NaN,0
2,8462,-137.0,119.0,1
3,11391,-159.0,NaN,0
4,23629,-47.0,NaN,0


In [ ]:
# Step 6: Create base table: one row per student with their final_result
# We use this as the starting point for our final dataset.

base = student_info[["id_student", "final_result"]].drop_duplicates()

print("Base table shape:", base.shape)
display(base.head())

# Now we merge engagement, performance, and registration features into 'base'
# We use 'how="left"' because we want to keep all students from 'base' even if some have missing activity/performance.

final_df = (
    base
    .merge(engagement, on="id_student", how="left")
    .merge(performance, on="id_student", how="left")
    .merge(registration, on="id_student", how="left")
)

print("Final merged dataset shape (before filling missing):", final_df.shape)
display(final_df.head())


Base table shape: (30749, 2)


,id_student,final_result
0,11391,Pass
1,28400,Pass
2,30268,Withdrawn
3,31604,Pass
4,32885,Pass


Final merged dataset shape (before filling missing): (30749, 12)


,id_student,final_result,total_clicks,active_days,avg_daily_clicks,first_active_day,last_active_day,quizzes_attempted,avg_quiz_score,date_registration,date_unregistration,unregistered_flag
0,11391,Pass,934.0,40.0,4.765306,-5.0,253.0,5.0,82.0,-159.0,NaN,0
1,28400,Pass,1435.0,80.0,3.337209,-10.0,239.0,5.0,66.4,-53.0,NaN,0
2,30268,Withdrawn,281.0,12.0,3.697368,-10.0,12.0,NaN,NaN,-92.0,12.0,1
3,31604,Pass,2158.0,123.0,3.254902,-10.0,264.0,5.0,76.0,-52.0,NaN,0
4,32885,Pass,1034.0,70.0,2.937500,-10.0,247.0,5.0,54.4,-176.0,NaN,0


In [ ]:
# Step 7: Handle missing values and save the cleaned dataset

# Fill missing numeric columns with 0
numeric_cols = final_df.select_dtypes(include="number").columns
final_df[numeric_cols] = final_df[numeric_cols].fillna(0)

print("Any remaining NaNs?")
print(final_df.isna().sum())

# Save to a CSV file that we'll use later for EDA and clustering
output_path = "lms_clean.csv"
final_df.to_csv(output_path, index=False)

print(f"\n✅ Saved cleaned LMS dataset to: {output_path}")
display(final_df.head())


Any remaining NaNs?
id_student             0
final_result           0
total_clicks           0
active_days            0
avg_daily_clicks       0
first_active_day       0
last_active_day        0
quizzes_attempted      0
avg_quiz_score         0
date_registration      0
date_unregistration    0
unregistered_flag      0
dtype: int64

✅ Saved cleaned LMS dataset to: lms_clean.csv


,id_student,final_result,total_clicks,active_days,avg_daily_clicks,first_active_day,last_active_day,quizzes_attempted,avg_quiz_score,date_registration,date_unregistration,unregistered_flag
0,11391,Pass,934.0,40.0,4.765306,-5.0,253.0,5.0,82.0,-159.0,0.0,0
1,28400,Pass,1435.0,80.0,3.337209,-10.0,239.0,5.0,66.4,-53.0,0.0,0
2,30268,Withdrawn,281.0,12.0,3.697368,-10.0,12.0,0.0,0.0,-92.0,12.0,1
3,31604,Pass,2158.0,123.0,3.254902,-10.0,264.0,5.0,76.0,-52.0,0.0,0
4,32885,Pass,1034.0,70.0,2.937500,-10.0,247.0,5.0,54.4,-176.0,0.0,0
